In [ ]:
# Copy the file from the local filesystem to DBFS
dbutils.fs.cp(
    'file:/Workspace/Users/thaith@smartosc.com/DE-training/DataBricks/Amazon Sale Report.csv',
    'dbfs:/Workspace/Users/thaith@smartosc.com/DE-training/DataBricks/Amazon Sale Report.csv'
)

# Load data to DataFrame
data = spark.read.format("csv").option("header", "true").load(
    "dbfs:/Workspace/Users/thaith@smartosc.com/DE-training/DataBricks/Amazon Sale Report.csv"
)

# Rename columns to remove invalid characters
for col in data.columns:
    new_col = col.replace(' ', '_').replace(';', '').replace('{', '').replace('}', '').replace('(', '').replace(')', '').replace('\n', '').replace('\t', '').replace('=', '')
    data = data.withColumnRenamed(col, new_col)

# Write data to Delta table
data.write.format("delta").mode("overwrite").save("dbfs:/delta/amazon_sale_report")

In [ ]:
# Read data from Delta table and display it
delta_table = spark.read.format("delta").load("dbfs:/delta/amazon_sale_report")
delta_table.display()

In [ ]:

# Remove column Unnamed:_22
data = data.drop("Unnamed:_22")

# Overwrite the Delta table with the updated DataFrame
data.write.format("delta").mode("overwrite").save("dbfs:/delta/amazon_sale_report")

In [ ]:
# Read data from Delta table and display it
display(delta_table)

In [ ]:
%fs ls 'dbfs:/delta/amazon_sale_report/_delta_log'

In [ ]:

%fs head 'dbfs:/delta/amazon_sale_report/_delta_log/00000000000000000001.json'

In [ ]:

# Create a table in the demo_databricks database using the delta_table DataFrame
delta_table.write.format("delta").mode("overwrite").saveAsTable("demo_databricks.default.delta_table")

In [ ]:
%sql
SELECT * FROM demo_databricks.default.delta_table LIMIT 10

In [ ]:
%sql
DESCRIBE HISTORY delta_table

In [ ]:
%sql
UPDATE delta_table
SET Amount = 0.0
WHERE Amount IS NULL;

SELECT * FROM delta_table LIMIT 10

In [ ]:

%sql
CREATE VIEW DressCategoryView AS
SELECT *
FROM delta_table
WHERE Category LIKE '%Dress%'

In [ ]:
%sql
SELECT * FROM DressCategoryView LIMIT 10

In [ ]:
%sql
SHOW TABLES

In [ ]:
%sql
DROP TABLE